# Code Overview:
* Get urls from target and append them into an empty array.
* Loop through each url in list to get data.
* Send data to nested dictionary so it can be converted into json data.
* Populate file with json data results.

### First let's start off by importing the modules we'll be using.

In [1]:
import requests
import json
from bs4 import BeautifulSoup as bs

### Set the target urls you'd like to scrape.

In [2]:
url = 'https://www.epmag.com/jobs/warehouse-inventory-auditor-73?page='
href_og = 'https://www.epmag.com'

### Make an empty array that will later be populated with  urls from each listing on the page.

In [3]:
url_list = []

### We can get started on building out a function that will populate our list with urls.

In [4]:
def get_url():
    
    # variable pages is just to specify how many pages you want.
    pages = raw_input('How many pages would you like to scrape?: ')
    
    # I'll make a loop that will go through each page specified.
    for page in xrange(int(pages)):
        r = requests.get(url + str(page))
        
        # parse the page with BeautfulSoup
        soup = bs(r.content, 'lxml')
        
        # we now that we have our parsed page lets define our links.
        links1 = soup.find_all('div', {'class' : 'views-row views-row-1 views-row-odd views-row-first'})
        links2 = soup.find_all('div', {'class' : 'views-row views-row-2 views-row-even'})
        links3 = soup.find_all('div', {'class' : 'views-row views-row-3 views-row-odd'})
        links4 = soup.find_all('div', {'class' : 'views-row views-row-4 views-row-even'})
        links5 = soup.find_all('div', {'class' : 'views-row views-row-5 views-row-odd'})
        links6 = soup.find_all('div', {'class' : 'views-row views-row-6 views-row-even'})
        links7 = soup.find_all('div', {'class' : 'views-row views-row-7 views-row-odd'})
        links8 = soup.find_all('div', {'class' : 'views-row views-row-8 views-row-even'})
        links9 = soup.find_all('div', {'class' : 'views-row views-row-9 views-row-odd'})
        links10 = soup.find_all('div', {'class' : 'views-row views-row-10 views-row-even views-row-last'})
        
        # lets make an array for our links
        links = [links1, links2, links3, links4, links5, links6, links7, links8, links9, links10]
        
        
        
        # iterate through our array and append each url to our empty array url_list
        for x in xrange(10):
            for link in links[x - 1]:
                href = link.find_all('a')
                for x in href:
                    url_list.append( href_og + str(x.get('href')))
                    
        # just to let us know the code is working we'll print 'done' after each successful scrape
        print('Done')

### Lets run our function to see if it works.

In [5]:
get_url()

How many pages would you like to scrape?: 5
Done
Done
Done
Done
Done


### Check our list

In [18]:
print('There are ' + str(len(url_list)) + " urls in your list.")
print("With 10 urls per page you have scraped " + str(len(url_list) / 10) + " pages." + "\n")
print("Here are the first 5 urls:")
for x in xrange(5): print(url_list[x])

There are 50 urls in your list.
With 10 urls per page you have scraped 5 pages.

Here are the first 5 urls:
https://www.epmag.com/jobs/planning-engineer-31
https://www.epmag.com/jobs/senior-contract-specialist
https://www.epmag.com/jobs/chemical-engineer-5
https://www.epmag.com/jobs/electronics-engineer-pcb-design-engineer
https://www.epmag.com/jobs/associate-materials-control-specialist-1


### Next function will make a get requests to each url in url_list and scrape the data we need.

In [6]:
def get_data():
    
    # Define a dictionary that we will populate later with or data
    data_json = {}
    
    # Loop through url_list and get the details you need per url.
    for url in range(len(url_list)):
        r = requests.get(url_list[url])
        soups = BeautifulSoup(r.content, 'lxml')
        title = soups.find_all('h1', {'class' : 'page-header'})
        company = soups.find_all('div', {'class' : 'field field-name-field-job-company field-type-text field-label-above'})
        description = soups.find_all('div', {'id' : 'job-description'})
        location = soups.find_all('div', {'class' : 'field field-name-field-job-location field-type-text field-label-above'})
        industry = soups.find_all('div', {'class' : 'field field-name-field-job-category field-type-text field-label-above'})
        rate = soups.find_all('div', {'class' : 'field field-name-field-job-salary field-type-text field-label-above'})
        frequency = soups.find_all('div', {'class' : 'field field-name-field-job-jobtype field-type-text field-label-above'})
        
        data_list = []
        
        for x in title:
            titles = x.text

        for x in company:
            company = x.text
            data_list.append({'company' : company.replace('\n', '').encode('ascii','ignore').replace('Company', '')})

        for x in description:
            description = x.text
            data_list.append({'description' : description.replace('\n', '').encode('ascii','ignore')})

        for x in location:
            location = x.text
            data_list.append({'location' : location.replace('\n', '').encode('ascii','ignore').replace('Location', '')})
            
        for x in industry:
            industry = x.text
            data_list.append({'industry' : industry.replace('\n', '').encode('ascii','ignore').replace('Category', '')})
            
        for x in rate:
            rate = x.text
            data_list.append({'rate' : rate.replace('\n', '').encode('ascii','ignore').replace('Salary', '')})
            
        for x in frequency:
            frequency = x.text
            data_list.append({'frequency' : frequency.replace('\n', '').encode('ascii','ignore').replace('Job Type', '')})
            
        # Turn data into json format and write your results to a json file.
        data_json[str(titles.encode('ascii','ignore'))] = data_list
        file = open('epmag.json','w')
        json.dump(data_json, file, indent=2)
    file.close()
    

## Changes to I make this better?

* Adding a celery worker.
* Creating object instead of two funtions.
* Store data into mysql database or s3 bucket and pull from there.
* Minimize the amount of lines of code used.